In [ ]:
import streamlit as st
import asyncio
import sys
from contextlib import contextmanager
from io import StringIO
from threading import current_thread
from streamlit.runtime.scriptrunner import add_script_run_ctx
from your_finance_agent_module import run_finance_agent

st.set_page_config(layout="wide", page_title="Finance Agent")

@contextmanager
def st_redirect(src, dst):
    placeholder = st.empty()
    output_func = getattr(placeholder, dst)

    with StringIO() as buffer:
        old_write = src.write

        def new_write(b):
            if add_script_run_ctx():
                buffer.write(b)
                output_func(buffer.getvalue())
            else:
                old_write(b)

        try:
            src.write = new_write
            yield
        finally:
            src.write = old_write

@contextmanager
def st_stdout(dst):
    with st_redirect(sys.stdout, dst):
        yield

@contextmanager
def st_stderr(dst):
    with st_redirect(sys.stderr, dst):
        yield

st.title("Finance Agent Query Interface")

# Sidebar for query input
with st.sidebar:
    st.header("Query Input")
    query = st.text_area("Enter your finance-related query:", height=100)
    run_button = st.button("Run Query", type="primary")

# Main content area
if run_button and query:
    st.subheader("Process Log")
    log_container = st.container()
    
    result_container = st.container()

    with log_container:
        with st_stdout("code"):
            result = asyncio.run(run_finance_agent(query))

    with result_container:
        st.success("Query processing completed!")
        
        # Display graph database results if available
        if 'graph_db_result' in result:
            st.subheader("Graph Database Results")
            st.json(result['graph_db_result'])
        
        # Display expanded queries if available
        if 'expanded_queries' in result:
            st.subheader("Expanded Queries")
            for i, eq in enumerate(result['expanded_queries'], 1):
                st.write(f"{i}. {eq}")
        
        # Display subgraph results if available
        if 'subgraph_results' in result:
            st.subheader("Subgraph Results")
            for subgraph, subresult in result['subgraph_results'].items():
                with st.expander(f"Results from {subgraph}"):
                    st.json(subresult)
        
        # Display final response
        st.subheader("Final Response")
        st.write(result.get('final_response', 'No final response available.'))

else:
    st.info("Enter a query in the sidebar and click 'Run Query' to start processing.")

# Footer
st.markdown("---")
st.markdown("© 2023 Finance Agent. All rights reserved.")